In [83]:
import gym
import random
import numpy as np
from gym import Env
from gym.spaces import Discrete,Box,MultiDiscrete,Tuple
import random
import time

import matplotlib.pyplot as plt

In [128]:
class microgrid(Env):
    
    
    def __init__(self):
        
        self.D = [800, 850, 880, 900, 860, 930, 950] 
            #gen_n=[p_low, p_high, a, b, c, p_r]
        self.gen_1  =[200, 600, 0.002, 10, 500, 50]
        self.gen_2 = [100, 400, 0.0025, 8, 300, 50]
        self.gen_3 = [100, 300, 0.0050, 6, 100, 50]
        self.gen_4 = [50, 200, 0.0060,  5, 90, 50]
        
        
        self.action_space = MultiDiscrete([400,300,200,150])
        #self.action_space= Discrete(5)
        self.observation_space= Discrete(5)
        #self.observation_space = MultiDiscrete([400,300,200,150])
        self.state_length = len(self.D)
        
        self.index=0
        
        
    def step(self, action):
        

        self.power_supply = 800  
        #self.power_supply = sum(action)
        self.state = action
        self.power_demand = self.D[self.index]
        self.index +=1
        
        
        if self.power_supply == self.power_demand:
            reward = 1
            
        else:
            reward =-1
        
        # optional information 
        info = {}
        
        self.state_length -=1
        if self.state_length ==0:
            done = True
        else:
            done = False
            
        return self.state, reward, done, info 
    
    def render(self):
        
        pass
    def reset(self):
        # reset the i
        self.index=0
        self.state_length = len(self.D)

    

In [81]:
'''env=microgrid()
done = False
import warnings
warnings.filterwarnings('ignore')

while not done:
    action= 100,0,0,0
    new_state,reward,done, info= env.step(action)
    print(env.power_demand)
action = env.action_space.sample()
print(action)    
print(action+[1,1,50,10])
print(env.gen_1[1])
print(type(env.action_space.nvec))
q_table= np.zeros((5,5))
print(q_table) '''

"env=microgrid()\ndone = False\nimport warnings\nwarnings.filterwarnings('ignore')\n\nwhile not done:\n    action= 100,0,0,0\n    new_state,reward,done, info= env.step(action)\n    print(env.power_demand)\naction = env.action_space.sample()\nprint(action)    \nprint(action+[1,1,50,10])\nprint(env.gen_1[1])\nprint(type(env.action_space.nvec))\nq_table= np.zeros((5,5))\nprint(q_table)"

In [162]:
env=microgrid()

cost = set() 
def get_feasible_action(action,damand):
    sum = 0
    for i in action :
        sum += i
    if sum == damand:
        return True
    else:
        return False

def get_cost(action):
    cost=((2**env.gen_1[2])*action[0]+ env.gen_1[3]*action[0]+ env.gen_1[4]) +  \
         ((2**env.gen_2[2])*action[1]+ env.gen_2[3]*action[1]+ env.gen_2[4]) + \
         ((2**env.gen_3[2])*action[2]+ env.gen_3[3]*action[2]+ env.gen_3[4])+  \
         ((2**env.gen_4[2])*action[3]+ env.gen_4[3]*action[3]+ env.gen_4[4])
    return cost

temp={}
ep =100000
for n in range(ep):
    action = env.action_space.sample()+ [200,100,100,50]
    feasible = get_feasible_action(action,800)
    if feasible:
        c= get_cost(action)
        cost.add(c)
        temp[str(c)]=str(action)
        




In [165]:
min_cost = min(cost)

print(temp[str(min_cost)])

[202 101 298 199]


In [166]:
print(min(cost))

7403.3193192077815


In [145]:
env=microgrid()

q_table= np.zeros((5,5))


def get_cost(action):
    cost=((2**env.gen_1[2])*action[0]+ env.gen_1[3]*env.action[0]+ env.gen_1[4]) +  \
         ((2**env.gen_2[2])*action[0]+ env.gen_2[3]*env.action[0]+ env.gen_2[4]) + \
         ((2**env.gen_3[2])*action[0]+ env.gen_3[3]*env.action[0]+ env.gen_3[4])+  \
         ((2**env.gen_4[2])*action[0]+ env.gen_4[3]*env.action[0]+ env.gen_4[4])
    return cost


num_episodes =10000


learning_rate = 0.1
discount_rate = 0.99
exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001 

rewards_all_episodes =[]
# Q-learning
for episode in range(num_episodes):
    state=env.reset()
    done= False
    rewards_current_episode = 0
    
    while not done:
        
        # Exploration-exploitation trade off 
        exploration_rate_threshole =random.uniform(0,1)
        if exploration_rate_threshole > exploration_rate:
            action = np.argmax(q_table[state, : ])
        else:
            action = env.action_space.sample()
        
        new_state, reward , done , info = env.step(action)
        
        # Update Q-table for Q(s,a)
        q_table[state,action] = q_table[state,action] * (1- learning_rate)+ \
                                learning_rate * (reward + discount_rate *np.max(q_table[new_state, :]))
        state =new_state
        rewards_current_episode += reward

    
    
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    rewards_all_episodes.append(rewards_current_episode)
    

rewards_per_thosand_episodes =np.split(np.array(rewards_all_episodes), num_episodes/1000)
count=1000

for r in rewards_per_thosand_episodes :
    print(count, " : ", str(sum(r/1000)))
    count += 1000
    


IndexError: index 252 is out of bounds for axis 0 with size 5

D[-1]

In [119]:
q_table= np.zeros((5,5))
print(q_table)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


600


TypeError: 'numpy.int64' object is not callable